In [1]:
# https://plotly.com/~jackp/17421/plotly-candlestick-chart-in-python/#/

import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas as pd
from multiprocessing import Pool
import tqdm
import gc
import Utils
import funtions

In [2]:
nasdaq = Utils.get_nasdaq_tickers()
len(nasdaq.Symbol)

5900

In [3]:
# Get today's date
t = datetime.now().strftime('%m%d%Y')

In [4]:
results = []

In [5]:
%%time

n_cpu = 4

with Pool(processes=n_cpu) as p:
    max_ = len(nasdaq.Symbol[400:500])
    with tqdm.tqdm_notebook(total=max_) as pbar: # May need to change the tqdm_notebook later
        for i, r in enumerate(p.imap(funtions.All_Min_and_Max, nasdaq.Symbol[400:500])): 
            # Default time frame from 01/02/2020 to now
            results.append(r)
            pbar.update(1)


CPU times: user 188 ms, sys: 312 ms, total: 500 ms
Wall time: 11.4 s


In [6]:
len(results)

100

In [7]:
results1 = [l for l in results if l != None]

results1 = pd.DataFrame(results1,
                        columns=[
                            'Symbol', 'MaxPriceDate', 'MaxPrice',
                            'MinPriceDate', 'MinPrice', 'LastDate',
                            'LastPrice', 'PrctHighToLow', 'PrctHightToCurrent'
                        ])

In [8]:
path = r'~/stock_analysis/Analyses/'
filename = "MinMaxAnalysis" + t + ".csv"
results1.to_csv(path+filename)